## Download and extract the CIFAR10 dataset
More info here: https://www.cs.toronto.edu/~kriz/cifar.html

In [1]:
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz

--2018-11-30 15:05:03--  https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M  4.46MB/s    in 50s     

2018-11-30 15:05:53 (3.26 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]



In [2]:
!echo Extracting data
!tar xf cifar-10-python.tar.gz
!echo Done extracting

Extracting data
Done extracting


## Load the images and convert to a format for TensorFlow

In [3]:
import pickle
import numpy as np
from keras.utils import to_categorical


def cifar_to_2d(images):
    images = np.array(images, dtype='float') / 255
    images = images.reshape([-1, 3, 32, 32])  # Convert raw data to 2D images with 3 channels
    images = images.transpose(0, 2, 3, 1)  # Put the channels dimension last (TensorFlow requirement)
    return images


train_X = np.zeros(shape=[50000, 32, 32, 3], dtype=float)  # Placeholder for 50k images, 32x32 with 3 channels
train_y = np.zeros(shape=[50000, 10], dtype=int)  # 1-hot encoding 10 classes
for i in range(1, 6):
    print('Loading training set ' + str(i))
    with open('cifar-10-batches-py/data_batch_' + str(i), 'rb') as infile:
        img_dict = pickle.load(infile, encoding='bytes')
        train_X[(i - 1) * 10000:i * 10000, :] = cifar_to_2d(img_dict[b'data'])
        train_y[(i - 1) * 10000:i * 10000, :] = to_categorical(img_dict[b'labels'])

print('Loading testing set')
with open('cifar-10-batches-py/test_batch', 'rb') as infile:
    img_dict = pickle.load(infile, encoding='bytes')
    test_X = cifar_to_2d(img_dict[b'data'])
    test_y = to_categorical(img_dict[b'labels'])


# Choose a random 10% of training data to use as the validation set.
np.random.seed(11798)
val_i = np.random.choice(len(train_X), 5000, replace=False)
val_X = train_X[val_i]
val_y = train_y[val_i]
train_X = train_X[~np.isin(np.arange(len(train_X)), val_i)]
train_y = train_y[~np.isin(np.arange(len(train_y)), val_i)]

print(val_X.shape)  # Should now be 5000 32x32 images with 3 channels
print(val_y.shape)  # Should now be 5000 one-hot labels with 10 classes
print(train_X.shape)  # Should now be 45000 32x32 images with 3 channels
print(train_y.shape)  # Should now be 45000 one-hot labels with 10 classes

Using TensorFlow backend.


Loading training set 1
Loading training set 2
Loading training set 3
Loading training set 4
Loading training set 5
Loading testing set
(5000, 32, 32, 3)
(5000, 10)
(45000, 32, 32, 3)
(45000, 10)


## Design a convolutional model for classifying the images

In [29]:
from keras import models, layers, optimizers


# Here we design a model using the "functional" Keras API.
m_input = layers.Input(shape=train_X[0].shape)  # Use the shape of the first image as input shape.
m = m_input

#m = layers.Dropout(rate=0.2)(m) # using dropout here will not improve


m = layers.Conv2D(filters=100,kernel_size=(2, 2))(m)  # https://keras.io/layers/convolutional/#conv2d




#m = layers.convolutional.SeparableConv2D(filters=100,kernel_size=(2,2))(m)  # this method  is not better than above


m = layers.Activation('relu')(m)


m = layers.advanced_activations.LeakyReLU(alpha=0.35)(m)# this layer helped a litte




m = layers.MaxPooling2D(pool_size=(2, 2))(m)  # https://keras.io/layers/pooling/



m = layers.Flatten()(m) 



m = layers.Dense(50, activation='relu')(m)# used to be 60: 92.96%  After several tests, I found out that 50 is the best with result of 93.14%


m = layers.Dropout(rate=0.1)(m)   #  92.8%->93.14%: using dropout here increase 0.3%



m = layers.Dense(10, activation='softmax')(m) # activation used to be 'relu'. I found out "softmax" is better. "Softmax" is also better than "sigmoid"  





m_output = m

model = models.Model(m_input, m_output)

#opt = optimizers.RMSprop(lr=.001)  # 34.24
#opt = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06) # 0.001 is the best lr.  and accuracy rate is 93.14%
#opt = optimizers.Adagrad(lr=.001)  # 91.27%
#opt = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-06) # 93.18%  √
opt = optimizers.Adamax(lr=0.003, beta_1=0.9, beta_2=0.999, epsilon=1e-08) # 0.003: 93.29%
#opt = optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004) # 93.07

# I tested out all the optimizers above and found out Adamax is the best with lr at 0.002.


model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])  #   loss used to be "categorical_crossentropy" and I changed it to "binary_crossentropy" this increased the accuracy rate a lot.

# The accuracy rate in the end is 93.12%

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 31, 31, 100)       1300      
_________________________________________________________________
activation_9 (Activation)    (None, 31, 31, 100)       0         
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 31, 31, 100)       0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 15, 15, 100)       0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 22500)             0         
_________________________________________________________________
dense_17 (Dense)             (None, 50)                1125050   
__________

## Train and validate the model
All your hyperparameter tuning needs to be done here!

In [30]:
model.fit(train_X, train_y, batch_size=128, epochs=10, validation_data=(val_X, val_y))

Train on 45000 samples, validate on 5000 samples
Epoch 1/10
45000/45000 [==============================] - 12s 268us/step - loss: 0.2661 - acc: 0.9042 - val_loss: 0.2268 - val_acc: 0.9132
Epoch 2/10
45000/45000 [==============================] - 11s 249us/step - loss: 0.2219 - acc: 0.9149 - val_loss: 0.2054 - val_acc: 0.9203
Epoch 3/10
45000/45000 [==============================] - 11s 247us/step - loss: 0.2047 - acc: 0.9207 - val_loss: 0.1954 - val_acc: 0.9244
Epoch 4/10
45000/45000 [==============================] - 11s 248us/step - loss: 0.1940 - acc: 0.9245 - val_loss: 0.1891 - val_acc: 0.9273
Epoch 5/10
45000/45000 [==============================] - 11s 248us/step - loss: 0.1851 - acc: 0.9280 - val_loss: 0.1802 - val_acc: 0.9305
Epoch 6/10
45000/45000 [==============================] - 11s 247us/step - loss: 0.1784 - acc: 0.9306 - val_loss: 0.1779 - val_acc: 0.9308
Epoch 7/10
45000/45000 [==============================] - 11s 248us/step - loss: 0.1729 - acc: 0.9327 - val_loss: 0.1

## Evaluate the FINAL model on testing data
Only run once, when you are finished adjusting your model

In [31]:
loss, acc = model.evaluate(test_X, test_y)
print('Test accuracy:', acc)

10000/10000 [==============================] - 2s 160us/step
Test accuracy: 0.9329599878311158


## Exporting your results to PDF
NOTE: This currently only seems to work in the Chrome browser

1. Download your notebook with _File -> Download .ipynb_
1. Rename with your name like in other assignments, for example bosch_nigel_assignment5b.ipynb
1. Upload to the _Files_ pane on the left side of the screen
1. Run the command in the next cell (with your filename) to convert to PDF
1. Click _Refresh_ in the Files pane on the left
1. Double-click the PDF to download it

In [0]:
!apt install pandoc
!apt install texlive-xetex
!jupyter nbconvert --to pdf Wang_Xiaoxin_assignment5b.ipynb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pandoc is already the newest version (1.19.2.4~dfsg-1build4).
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
texlive-xetex is already the newest version (2017.20180305-1).
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
[NbConvertApp] Converting notebook Wang_Xiaoxin_assignment5b.ipynb to pdf
[NbConvertApp] Writing 101832 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: [u'xelatex', u'./notebook.tex']
[NbConvertApp] Running bibtex 1 time: [u'bibtex', u'./notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 67491 bytes to Wang_Xiaoxin_assignment5b.pdf
